In [1]:
%load_ext gprof2dot_magic
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from base import SuperLearner, BMA, try_super_learners
import pandas as pd
import numpy as np
from pyearth import Earth
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # warnings from py-earth

v_folds = 5
ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=100,min_samples_split=5)
gbm = ensemble.GradientBoostingRegressor()
# earth is sort of like D/S/A?
earth=Earth(max_terms=50,max_degree=3,use_fast=True,verbose=0) # get this from https://github.com/scikit-learn-contrib/py-earth
rtree=DecisionTreeRegressor(max_depth=3,min_samples_split=5)

cands=[ols,lars,earth,rf]
metas=[ols,lasso,ridge,earth,rf,rtree,gbm]
def helper(X1,y1,X2,y2):
    display(try_super_learners(cands,metas,X1,y1,X2,y2))
#     sl=SuperLearner(cand_learners=cands,V=10,meta_learner=ols).fit(X1,y1)
#     for meta in metas:
#         sl.meta_learner_=meta.fit(sl.Z_train_cv_,y1)
#         df=sl.debug(X1,y1,X2,y2,skip_fit=True)
        
    display(BMA(cand_learners=cands).debug(X1,y1,X2,y2))

The gprof2dot_magic module is not an IPython extension.


c:\program files\python36\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# first simulation study
def sim1(n):
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

train1,test1=sim1(500),sim1(10000)
helper(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,6.457040,6.790669,6.245139
1,LarsCV,6.457040,6.829065,6.245138
2,Earth,1.030888,1.451809,1.138670
3,RandomForestRegressor,0.777849,2.307637,1.634932
4,Meta (LinearRegression),0.884734,1.390292,1.128914
5,Meta (LassoCV),0.888831,1.394042,1.124043
6,Meta (RidgeCV),0.883509,1.392304,1.127069
7,Meta (Earth),0.890154,1.393103,1.124968
8,Meta (RandomForestRegressor),1.043837,0.353166,1.328451
9,Meta (DecisionTreeRegressor),1.224555,1.321092,1.336481


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,6.457040,6.245139,5.000000e-01
1,LarsCV,6.457040,6.245138,5.000000e-01
2,Earth,1.030888,1.138670,3.115372e-200
3,RandomForestRegressor,0.790212,1.671886,3.815873e-229
4,BMA,6.457040,6.245139,NaN


In [3]:
diabetes=datasets.load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2)

helper(X_train,y_train,X_test,y_test)

c:\program files\python36\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,2821.817921,3086.047153,3053.236274
1,LarsCV,2872.620180,3085.741096,3120.528850
2,Earth,2670.971317,3696.622783,3522.458001
3,RandomForestRegressor,622.646501,3526.271519,3586.149295
4,Meta (LinearRegression),2533.784279,3064.294495,3086.133341
5,Meta (LassoCV),2576.703040,3065.777360,3104.815030
6,Meta (RidgeCV),2533.747745,3064.294496,3086.141010
7,Meta (Earth),2909.444297,2919.021873,2923.065366
8,Meta (RandomForestRegressor),2998.107004,738.284619,3121.581538
9,Meta (DecisionTreeRegressor),2704.168590,2741.844942,3284.623382


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,2821.817921,3053.236274,4.111752e-02
1,LarsCV,2872.620180,3120.528850,9.588800e-01
2,Earth,2670.971317,3522.458001,2.528010e-06
3,RandomForestRegressor,627.934480,3622.352372,2.647483e-117
4,BMA,2868.526687,3115.398256,NaN


In [4]:
pr=pd.read_csv("../CASP.csv")
feature_cols= pr.columns[pr.columns!='RMSD']
from sklearn.preprocessing import scale
pr.loc[:,feature_cols]=scale(pr.loc[:,feature_cols])

In [5]:
prtrain,prtest=train_test_split(pr.sample(1000))

In [7]:
helper(prtrain.loc[:,feature_cols],prtrain.RMSD,prtest.loc[:,feature_cols],prtest.RMSD)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,26.832330,27.881049,25.271404
1,LarsCV,27.083743,27.629469,25.626118
2,Earth,21.395883,32.104671,27.008900
3,RandomForestRegressor,4.785307,27.671291,23.968684
4,Meta (LinearRegression),12.793622,25.892323,23.183354
5,Meta (LassoCV),12.957361,25.894412,23.212740
6,Meta (RidgeCV),12.828720,25.892376,23.187997
7,Meta (Earth),13.860623,24.734130,22.869964
8,Meta (RandomForestRegressor),18.760823,6.419984,23.736597
9,Meta (DecisionTreeRegressor),20.144171,24.478836,22.802457


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,26.832330,25.271404,2.938900e-02
1,LarsCV,27.083743,25.626118,9.706110e-01
2,Earth,21.395883,27.008900,3.936602e-39
3,RandomForestRegressor,4.924800,23.687328,2.094240e-278
4,BMA,27.069182,25.609819,NaN
